In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
from tensorflow.keras import backend
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
os.environ['CUDA_VISIBLE_DEVICES'] = '/gpu:0'
# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("/home/xs/tensorflow_test/models/research/object_detection")
from object_detection.utils import ops as utils_opsjiqi

#if tf.__version__ < '1.4.0':
#  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')

from utils import label_map_util

from utils import visualization_utils as vis_util

In [2]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.compat.v1.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.compat.v1.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.compat.v1.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.compat.v1.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [3]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [4]:
# What model to download.
MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('/home/xs/tensorflow_test/models/research/object_detection/data', 'mscoco_label_map.pbtxt')

NUM_CLASSES = 90

detection_graph = tf.Graph()
with detection_graph.as_default():
  #od_graph_def = tf.GraphDef()
  od_graph_def = tf.compat.v1.GraphDef()
  with tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')
    
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)


In [ ]:
import cv2
import time

cap = cv2.VideoCapture(2)  #### choose camera id by  private computer
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 640)
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))

with detection_graph.as_default():
    with tf.compat.v1.Session() as sess:
    
        while True:
            time_start=time.time()
            ret, frame = cap.read()
            
            image_np_expanded = np.expand_dims(frame, axis=0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            # Actual detection.
            (num_detections,boxes, scores, classes) = sess.run(
                [num_detections, boxes, scores, classes],
                feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                frame, np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores), category_index,
                use_normalized_coordinates=True,
                line_thickness=8)
 
            cv2.imshow('SYCU object detection', cv2.resize(frame, (320, 320)))
            time_end=time.time()
            print('time cost',(time_end-time_start)*1000,'ms')
            if cv2.waitKey(25) & 0xFF == ord('q'):
                cv2.destroyAllWindows()
                break
                
cap.release()
cv2.destroyAllWindows()

time cost 1629.9617290496826 ms
time cost 31.77928924560547 ms
time cost 35.18223762512207 ms
time cost 32.593488693237305 ms
time cost 46.72956466674805 ms
time cost 31.909704208374023 ms
time cost 29.834747314453125 ms
time cost 29.153823852539062 ms
time cost 29.776573181152344 ms
time cost 29.90865707397461 ms
time cost 28.44691276550293 ms
time cost 31.18610382080078 ms
time cost 33.26559066772461 ms
time cost 29.531478881835938 ms
time cost 34.54089164733887 ms
time cost 33.83827209472656 ms
time cost 29.560089111328125 ms
time cost 29.961347579956055 ms
time cost 29.278278350830078 ms
time cost 31.226158142089844 ms
time cost 29.38079833984375 ms
time cost 29.889583587646484 ms
time cost 28.673410415649414 ms
time cost 31.09288215637207 ms
time cost 35.53009033203125 ms
time cost 33.324480056762695 ms
time cost 32.26208686828613 ms
time cost 31.250476837158203 ms
time cost 32.07826614379883 ms
time cost 29.331445693969727 ms
time cost 35.43353080749512 ms
time cost 31.1245918273

time cost 30.673742294311523 ms
time cost 30.07340431213379 ms
time cost 28.730154037475586 ms
time cost 28.77664566040039 ms
time cost 31.080961227416992 ms
time cost 29.991626739501953 ms
time cost 28.171062469482422 ms
time cost 28.72180938720703 ms
time cost 30.38167953491211 ms
time cost 28.431415557861328 ms
time cost 29.929637908935547 ms
time cost 27.930259704589844 ms
time cost 28.611183166503906 ms
time cost 33.406734466552734 ms
time cost 33.53118896484375 ms
time cost 29.676437377929688 ms
time cost 29.89363670349121 ms
time cost 30.20501136779785 ms
time cost 29.17337417602539 ms
time cost 28.215408325195312 ms
time cost 31.079769134521484 ms
time cost 28.32770347595215 ms
time cost 29.69813346862793 ms
time cost 29.93607521057129 ms
time cost 29.161453247070312 ms
time cost 27.434825897216797 ms
time cost 31.88037872314453 ms
time cost 29.173851013183594 ms
time cost 28.721094131469727 ms
time cost 30.287981033325195 ms
time cost 28.609037399291992 ms
time cost 30.3215980

time cost 32.58466720581055 ms
time cost 31.577587127685547 ms
time cost 28.6102294921875 ms
time cost 30.53903579711914 ms
time cost 29.898643493652344 ms
time cost 35.77852249145508 ms
time cost 31.615018844604492 ms
time cost 31.041860580444336 ms
time cost 29.462575912475586 ms
time cost 29.68740463256836 ms
time cost 28.493165969848633 ms
time cost 29.13951873779297 ms
time cost 30.735254287719727 ms
time cost 30.922889709472656 ms
time cost 30.794382095336914 ms
time cost 29.128551483154297 ms
time cost 36.03935241699219 ms
time cost 36.20195388793945 ms
time cost 31.448841094970703 ms
time cost 32.639503479003906 ms
time cost 35.973310470581055 ms
time cost 31.416893005371094 ms
time cost 36.07058525085449 ms
time cost 33.75530242919922 ms
time cost 33.67042541503906 ms
time cost 32.86409378051758 ms
time cost 31.733274459838867 ms
time cost 34.75236892700195 ms
time cost 33.277273178100586 ms
time cost 31.075716018676758 ms
time cost 32.61256217956543 ms
time cost 35.4664325714

time cost 30.685901641845703 ms
time cost 30.98583221435547 ms
time cost 32.15742111206055 ms
time cost 31.955957412719727 ms
time cost 30.77983856201172 ms
time cost 29.976844787597656 ms
time cost 33.41794013977051 ms
time cost 31.481027603149414 ms
time cost 35.00795364379883 ms
time cost 30.465364456176758 ms
time cost 28.374433517456055 ms
time cost 28.883695602416992 ms
time cost 31.548023223876953 ms
time cost 31.862497329711914 ms
time cost 31.314373016357422 ms
time cost 30.82752227783203 ms
time cost 31.524181365966797 ms
time cost 30.81059455871582 ms
time cost 29.444456100463867 ms
time cost 29.662132263183594 ms
time cost 33.30802917480469 ms
time cost 32.64331817626953 ms
time cost 30.551910400390625 ms
time cost 34.15727615356445 ms
time cost 32.80901908874512 ms
time cost 34.343719482421875 ms
time cost 33.80584716796875 ms
time cost 28.252124786376953 ms
time cost 36.098480224609375 ms
time cost 31.138896942138672 ms
time cost 32.40489959716797 ms
time cost 35.03251075

time cost 30.238628387451172 ms
time cost 29.484987258911133 ms
time cost 36.214351654052734 ms
time cost 30.628442764282227 ms
time cost 31.029224395751953 ms
time cost 28.4879207611084 ms
time cost 28.885364532470703 ms
time cost 27.61054039001465 ms
time cost 30.60603141784668 ms
time cost 33.12563896179199 ms
time cost 30.75885772705078 ms
time cost 33.10656547546387 ms
time cost 31.23617172241211 ms
time cost 30.56621551513672 ms
time cost 29.691457748413086 ms
time cost 31.285524368286133 ms
time cost 32.89532661437988 ms
time cost 31.213998794555664 ms
time cost 30.976057052612305 ms
time cost 33.66208076477051 ms
time cost 31.1124324798584 ms
time cost 29.432058334350586 ms
time cost 36.792755126953125 ms
time cost 32.02509880065918 ms
time cost 32.19151496887207 ms
time cost 30.994176864624023 ms
time cost 32.502174377441406 ms
time cost 35.42494773864746 ms
time cost 30.490398406982422 ms
time cost 31.068801879882812 ms
time cost 34.093618392944336 ms
time cost 31.63194656372

time cost 32.53293037414551 ms
time cost 31.30793571472168 ms
time cost 29.300689697265625 ms
time cost 28.116941452026367 ms
time cost 32.84788131713867 ms
time cost 31.010150909423828 ms
time cost 31.287193298339844 ms
time cost 31.517982482910156 ms
time cost 33.74147415161133 ms
time cost 25.569677352905273 ms
time cost 28.866291046142578 ms
time cost 29.10161018371582 ms
time cost 29.60824966430664 ms
time cost 28.445720672607422 ms
time cost 29.014110565185547 ms
time cost 28.784513473510742 ms
time cost 29.982805252075195 ms
time cost 30.99966049194336 ms
time cost 28.841495513916016 ms
time cost 30.0142765045166 ms
time cost 29.59275245666504 ms
time cost 30.186891555786133 ms
time cost 32.13071823120117 ms
time cost 31.397104263305664 ms
time cost 29.98042106628418 ms
time cost 28.576135635375977 ms
time cost 27.581453323364258 ms
time cost 31.049728393554688 ms
time cost 32.38701820373535 ms
time cost 32.40847587585449 ms
time cost 32.66000747680664 ms
time cost 31.9881439208

time cost 32.94539451599121 ms
time cost 30.709505081176758 ms
time cost 34.07573699951172 ms
time cost 30.637025833129883 ms
time cost 34.75809097290039 ms
time cost 30.73906898498535 ms
time cost 31.908512115478516 ms
time cost 31.30364418029785 ms
time cost 32.50837326049805 ms
time cost 30.352115631103516 ms
time cost 33.391475677490234 ms
time cost 31.13865852355957 ms
time cost 31.450510025024414 ms
time cost 32.675981521606445 ms
time cost 32.559871673583984 ms
time cost 31.284570693969727 ms
time cost 34.581899642944336 ms
time cost 29.721498489379883 ms
time cost 33.06865692138672 ms
time cost 30.681133270263672 ms
time cost 30.344009399414062 ms
time cost 31.212329864501953 ms
time cost 33.43844413757324 ms
time cost 31.382322311401367 ms
time cost 31.486988067626953 ms
time cost 30.524253845214844 ms
time cost 33.85186195373535 ms
time cost 31.290769577026367 ms
time cost 30.30109405517578 ms
time cost 30.071258544921875 ms
time cost 32.42206573486328 ms
time cost 30.9097766

time cost 27.454137802124023 ms
time cost 30.333757400512695 ms
time cost 26.690006256103516 ms
time cost 31.02397918701172 ms
time cost 33.57529640197754 ms
time cost 27.60028839111328 ms
time cost 26.71980857849121 ms
time cost 27.77099609375 ms
time cost 26.637792587280273 ms
time cost 27.10890769958496 ms
time cost 27.43697166442871 ms
time cost 28.449535369873047 ms
time cost 26.760339736938477 ms
time cost 25.912046432495117 ms
time cost 27.481555938720703 ms
time cost 28.321027755737305 ms
time cost 28.59783172607422 ms
time cost 27.734041213989258 ms
time cost 26.952266693115234 ms
time cost 25.348663330078125 ms
time cost 28.809785842895508 ms
time cost 28.1982421875 ms
time cost 26.971817016601562 ms
time cost 28.41949462890625 ms
time cost 29.575347900390625 ms
time cost 28.00917625427246 ms
time cost 26.30162239074707 ms
time cost 26.919126510620117 ms
time cost 29.50906753540039 ms
time cost 30.632972717285156 ms
time cost 27.480363845825195 ms
time cost 32.743215560913086